### Notebook analysis the data to get a first hand impression

In [106]:
import pandas as pd
import os, sys
import pyarrow.parquet as pq
from pyarrow import csv
from calendar import monthrange
from google.cloud.storage import Client
from google.cloud import storage
import fsspec
import gcsfs
from dotenv import load_dotenv

Data is available from 2022-01-03 - 2022-04-25

In [107]:
load_dotenv()

True

In [179]:
data = r'../data/xetra/2022-04-22/2022-04-22_BINS_XETR07.csv'

In [180]:
df = pd.read_csv(data)

In [181]:
df.shape

(16054, 14)

In [195]:
df.isnull().sum()

ISIN              0
Mnemonic          0
SecurityDesc      0
SecurityType      0
Currency          0
SecurityID        0
Date              0
Time              0
StartPrice        0
MaxPrice          0
MinPrice          0
EndPrice          0
TradedVolume      0
NumberOfTrades    0
dtype: int64

In [182]:
df.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-04-22,07:00,16.450,16.45,16.450,16.45,5,1
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-04-22,07:00,40.980,40.98,40.620,40.82,1643,10
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-04-22,07:00,15.855,16.05,15.855,15.97,59288,86
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-04-22,07:00,196.550,196.75,195.750,196.35,2989,35
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-04-22,07:00,28.030,28.03,28.030,28.03,236,1


In [196]:
df.dtypes

ISIN               object
Mnemonic           object
SecurityDesc       object
SecurityType       object
Currency           object
SecurityID          int64
Date               object
Time               object
StartPrice        float64
MaxPrice          float64
MinPrice          float64
EndPrice          float64
TradedVolume        int64
NumberOfTrades      int64
dtype: object

In [12]:
# distribution of Mnenomic
df["Mnemonic"].value_counts()

Mnemonic
VOW3    60
DAI     60
IFX     60
LIN     60
BMW     60
        ..
V20D     1
MVEA     1
LV2B     1
XCO2     1
W8A      1
Name: count, Length: 1578, dtype: int64

In [13]:
df["SecurityType"].value_counts()

SecurityType
Common stock    6817
ETF             4905
ETN              186
ETC              165
Name: count, dtype: int64

In [14]:
df["Currency"].value_counts()

Currency
EUR    12069
USD        2
GBP        1
SEK        1
Name: count, dtype: int64

In [16]:
df["ISIN"].nunique()

1578

In [44]:
df["Time"].unique()

array(['10:00', '10:01', '10:02', '10:03', '10:04', '10:05', '10:06',
       '10:07', '10:08', '10:09', '10:10', '10:11', '10:12', '10:13',
       '10:14', '10:15', '10:16', '10:17', '10:18', '10:19', '10:20',
       '10:21', '10:22', '10:23', '10:24', '10:25', '10:26', '10:27',
       '10:28', '10:29', '10:30', '10:31', '10:32', '10:33', '10:34',
       '10:35', '10:36', '10:37', '10:38', '10:39', '10:40', '10:41',
       '10:42', '10:43', '10:44', '10:45', '10:46', '10:47', '10:48',
       '10:49', '10:50', '10:51', '10:52', '10:53', '10:54', '10:55',
       '10:56', '10:57', '10:58', '10:59'], dtype=object)

In [18]:
possible_columns_to_use = \
[
"ISIN",
"Date",
"Time",
"StartPrice",
"MaxPrice",
"MinPrice",
"EndPrice",
"TradedVolume"
]

In [20]:
arr = csv.read_csv(data)

In [21]:
arr.schema

ISIN: string
Mnemonic: string
SecurityDesc: string
SecurityType: string
Currency: string
SecurityID: int64
Date: date32[day]
Time: time32[s]
StartPrice: double
MaxPrice: double
MinPrice: double
EndPrice: double
TradedVolume: int64
NumberOfTrades: int64

In [198]:
arr.to_pandas().dtypes

ISIN               object
Mnemonic           object
SecurityDesc       object
SecurityType       object
Currency           object
SecurityID          int64
Date               object
Time               object
StartPrice        float64
MaxPrice          float64
MinPrice          float64
EndPrice          float64
TradedVolume        int64
NumberOfTrades      int64
dtype: object

In [24]:
def leap_year(year):
    """Check whether year is a leap year or not"""
    return (year%4 == 0) and not ( year%100 == 0) or year%400 == 0

In [36]:
leap_year(2023)

False

In [42]:
monthrange(2020, 2)

(5, 29)

In [115]:
# read parquet file from gcs
client = Client()
bucket = client.get_bucket("xetra-ds")
blob = bucket.get_blob("data/xetra/2022-01-03/2022-01-03_BINS_XETR08.parquet")
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 7: invalid start byte
# downloaded_file = blob.download_as_text(encoding="utf-8")

In [120]:
pd.read_csv(blob.download_as_string())

TypeError: Expected file path name or file-like object, got <class 'bytes'> type

In [111]:
def read_parquet(gs_directory_path, to_pandas=True):
    """
    Reads multiple (partitioned) parquet files from a GS directory
    e.g. 'gs://<bucket>/<directory>' (without ending /)
    """
    #gs = fsspec.filesystem("gcs", project='data-eng-375913')
    gs = gcsfs.GCSFileSystem(project=os.environ.get("GCP_PROJECT_ID"))
    arrow_df = pq.ParquetDataset(gs_directory_path, filesystem=gs)
    if to_pandas:
        return arrow_df.read().to_pandas()
    return arrow_df

In [113]:
# OSError: Forbidden: b/data/o
# de.datacamp@gmail.com does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist)
#dd = read_parquet("data/xetra/2022-01-03/2022-01-03_BINS_XETR08.parquet", to_pandas=True)

In [174]:
def list_blobs(bucket_name, prefix, delimiter=""):
    """Lists all the blobs in the bucket."""
    
    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    # Note: The call returns a response only when the iterator is consumed.
    num =0
    for blob in blobs:
        num += 1
        print(blob.name)
    print(num)

In [176]:
list_blobs("xetra-ds", "data/xetra/2022-04-22")

data/xetra/2022-04-22/2022-04-22_BINS_XETR07.parquet
data/xetra/2022-04-22/2022-04-22_BINS_XETR08.parquet
data/xetra/2022-04-22/2022-04-22_BINS_XETR09.parquet
data/xetra/2022-04-22/2022-04-22_BINS_XETR10.parquet
data/xetra/2022-04-22/2022-04-22_BINS_XETR11.parquet
data/xetra/2022-04-22/2022-04-22_BINS_XETR12.parquet
data/xetra/2022-04-22/2022-04-22_BINS_XETR13.parquet
data/xetra/2022-04-22/2022-04-22_BINS_XETR14.parquet
data/xetra/2022-04-22/2022-04-22_BINS_XETR15.parquet
9


In [121]:
gcs_path = "gs://xetra-ds/data/xetra/2022-01-03/2022-01-03_BINS_XETR08.parquet"
dd = pd.read_parquet(gcs_path)

In [122]:
dd.shape

(16961, 14)

In [123]:
dd.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-01-03,08:00:00,14.76,14.76,14.75,14.75,4414,2
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-01-03,08:00:00,37.64,37.66,37.60,37.66,1649,3
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-03,08:00:00,13.99,14.03,13.94,13.96,23011,36
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-01-03,08:00:00,180.00,180.05,179.50,179.50,2308,22
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-01-03,08:00:00,37.28,37.28,37.28,37.28,2897,1
